In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import scipy.misc as misc
import math
import imageio

# Project Imports
import llops as yp
import llops.operators as ops
from llops.solvers import objectivefunctions, regularizers, iterative
import llops.simulation as sim

# Set default backend and dtype
ops.setDefaultBackend('numpy') # arrayfire or numpy
ops.setDefaultDatatype('complex32')

# Generate Simple Forward Model

In [24]:
# Image size to simulate
image_size = np.array([128, 256])

# Create test object
x = sim.brain(image_size)

# Generate convolution kernel h
h_size = np.array([4, 4])
h = yp.zeros(image_size)
h[image_size[0] // 2 - h_size[0] // 2:image_size[0] // 2 + h_size[0] // 2,
  image_size[1] // 2 - h_size[1] // 2:image_size[1] // 2 + h_size[1] // 2] = yp.randn((h_size[0], h_size[1]))
h /= yp.sum(h)

# Generate Convolution Kernel
C = ops.Convolution(h)

# Generate Measurement
y = C * x

# Display Result
plt.figure()
plt.subplot(121)
plt.imshow(yp.abs(x))
plt.subplot(122)
plt.imshow(yp.abs(y))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
C.latex()

<IPython.core.display.Latex object>

In [25]:
C

[<C: (128, 256)x(128, 256) Convolution NUMPY operator with dtype=complex32>]

In [ ]:
yp.is

In [14]:
J.arguments['elements']

array([[0.942219  +0.j, 1.0046186 +0.j, 1.007985  +0.j, ...,
        0.97764397+0.j, 1.002749  +0.j, 1.0113766 +0.j],
       [1.0600861 +0.j, 0.9978062 +0.j, 0.9911985 +0.j, ...,
        1.0641291 +0.j, 1.0128345 +0.j, 1.0029546 +0.j],
       [1.0173366 +0.j, 1.0124779 +0.j, 1.0031376 +0.j, ...,
        1.0427309 +0.j, 0.95603675+0.j, 0.9669237 +0.j],
       ...,
       [1.0077353 +0.j, 1.0391666 +0.j, 0.9509164 +0.j, ...,
        1.0168316 +0.j, 1.0082844 +0.j, 1.0230091 +0.j],
       [1.0071073 +0.j, 1.0722712 +0.j, 1.0082763 +0.j, ...,
        1.0033957 +0.j, 1.0243622 +0.j, 1.0199288 +0.j],
       [1.0459895 +0.j, 1.0030985 +0.j, 1.0204476 +0.j, ...,
        0.98377657+0.j, 0.99668974+0.j, 0.95140725+0.j]], dtype=complex64)

In [16]:
('c', C, 'kernel'), ('j', J, 'elements')

(('c',
  <C: (128, 256)x(128, 256) Convolution NUMPY operator with dtype=complex32>,
  'kernel'),
 ('j',
  <d: (128, 256)x(128, 256) Diagonalize NUMPY operator with dtype=complex32>,
  'elements'))

In [6]:
from llops.operators import Operator

class ObjectiveFunction(Operator):
    """An objective function class. This class is mostly the same as an 
       llops.operators.Operator, but also supports gradients with respect 
       to multiple variables."""

    def __init__(self, operator_list, variable_set_function_list=None):

        # Convert operator_list to a list, if only a single operator is provided
        self.operator_list = operator_list
        self.variable_set_function_list = variable_set_function_list

        # Determine if this objective function is multivariate
        self.is_multivariate = type(operator_list) in [list, tuple]

        if self.is_multivariate:
            self.operator = self.operator_list[0]
            self.shape = [operator.shape for operator in self.operator_list]
        else:
            self.operator = self.operator_list
            self.shape = self.operator_list[0].shape




